# Word2Vec

In [ ]:
import time

import numpy as np
import tensorflow as tf

import utils_word2vec

# Leitura do dataset

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

with open('data/text8') as f: #wiki.pt-br.text
    text = f.read()

# Pré-processamento

In [3]:
words = utils_word2vec.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


Total de tokens e types

In [4]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


Transforma palavras em números inteiros. A palavra mais frequente recebe 0, a próxima recebe 1, e assim por diante.

In [5]:
vocab_to_int, int_to_vocab = utils_word2vec.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

# Subsampling (Sub-amostragem)

Remoção de palavras frequentes através de probabilidade. Remover ruídos, ter um treinamento mais rápido e melhorar as representações.

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

onde $t$ é um parâmetro limiar e $f(w_i)$ é a frequência da palavra $w_i$ no conjunto de dados

In [6]:
from collections import Counter
import random

threshold = 1e-5
word_counts = Counter(int_words)
total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

# Criando batches

Com o pré-processamento finalizado, é necessário pegar as palavras de contexto da palavra-alvo. Nesse caso, pegaremos 5 palavras anteriores a palavra-alvo e 5 posteriores.

In [7]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    
    return list(target_words)

Função que retorna lotes para a rede.

In [8]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

# Construindo o grafo

In [9]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')

In [10]:
n_vocab = len(int_to_vocab)
n_embedding = 200 # Number of embedding features 
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)

In [11]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocab))
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, 
                                      labels, embed,
                                      n_sampled, n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [12]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [13]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

mkdir: cannot create directory ‘checkpoints’: File exists


In [ ]:
epochs = 100
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/100 Iteration: 100 Avg. Training loss: 5.6608 0.1669 sec/batch
Epoch 1/100 Iteration: 200 Avg. Training loss: 5.6161 0.1590 sec/batch
Epoch 1/100 Iteration: 300 Avg. Training loss: 5.5197 0.1549 sec/batch
Epoch 1/100 Iteration: 400 Avg. Training loss: 5.6076 0.1512 sec/batch
Epoch 1/100 Iteration: 500 Avg. Training loss: 5.4958 0.1541 sec/batch
Epoch 1/100 Iteration: 600 Avg. Training loss: 5.5484 0.1603 sec/batch
Epoch 1/100 Iteration: 700 Avg. Training loss: 5.5599 0.1507 sec/batch
Epoch 1/100 Iteration: 800 Avg. Training loss: 5.5284 0.1454 sec/batch
Epoch 1/100 Iteration: 900 Avg. Training loss: 5.4973 0.1571 sec/batch
Epoch 1/100 Iteration: 1000 Avg. Training loss: 5.4726 0.1482 sec/batch
Nearest to new: acheson, medi, governmental, enthusiast, he, loft, oud, acknowledge,
Nearest to had: genotype, taxicab, aura, corbusier, drain, overstepped, maxell, incidence,
Nearest to their: objectivity, castigated, medan, withdraw, polygram, clitoridectomy, body, investigating,
Neares

In [15]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

In [16]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [17]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [18]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)

In [19]:
import os
with open(os.path.join('checkpoints', "metadata.tsv"), 'w') as f:
         for idx in range(len(int_to_vocab)):
            f.write(int_to_vocab[idx] + '\n')

In [20]:
from tensorflow.contrib.tensorboard.plugins import projector
summary_writer = tf.summary.FileWriter('checkpoints', sess.graph)
config = projector.ProjectorConfig()
embedding_conf = config.embeddings.add()
# embedding_conf.tensor_name = 'embedding:0'
embedding_conf.metadata_path = os.path.join('checkpoints', 'metadata.tsv')
projector.visualize_embeddings(summary_writer, config)